Implemented a Generator Class, which can generate vehicles and tasks for said vehicles with a poisson distribution.
The second to last cell can be manipulated to try out the FIFO/ EDF scheduling resource

In [62]:
! pip install simpy
import simpy
! pip install numpy
import numpy as np
import random
random.seed(42)     #does this effect numpy aswell?

In [ ]:
class Tester: 

    def dalle(self,environment):
        while True: 
         yield environment.timeout(1)
         print("image")

In [63]:
class Task:

    def __init__(self, id, priority, duration, vehicle):

        self.id = id
        self.priority = priority
        self.duration = duration
        self.vehicle = vehicle
        self.time_spent_processing = 0


    def process(self, environment, resource):

        print(f'Task {self.id} came into existence at {environment.now}')
        environment.plan.append(f'Task {self.id} came into existence at {environment.now} with priority {self.priority} and duration {self.duration}')
        
        # keep requesting resource if processing duration hasn't been reached
        while(self.time_spent_processing < self.duration):

             if isinstance(resource, simpy.PriorityResource):
                req = resource.request(priority = self.priority)
             else:
                req = resource.request()

             with req as req:
            
                try:
                    #before receiving resource
                    print(f'Task {self.id} requesting at {environment.now} with priority {self.priority}')
                    yield req
                    #while holding resource
                    print(f'Task {self.id} got resource at {environment.now}')
                    environment.plan.append(f'Task {self.id} got resource at {environment.now}')
                    #try to timeout for the time needed to complete the task
                    yield environment.timeout(self.duration - self.time_spent_processing) 
                    print(f'Task {self.id} completed at {environment.now}')
                    #exit loop if processing is complete
                    self.vehicle.tasks_completed.append(self)
                    environment.total_tasks_completed += 1
                    break

                except simpy.Interrupt as interrupt:
                    by = interrupt.cause.by
                    usage = environment.now - interrupt.cause.usage_since
                    #save the processing progress of the task
                    self.time_spent_processing += usage
                    print(f'Task {self.id} got preempted by {by} at {environment.now} after {usage} time entities')

In [64]:
class Vehicle:

    def __init__(self, id, environment, assigned_tasks, resource, lambda_tasks, lambda_dwell_time, generator):

        self.id = id
        self.environment = environment
        self.assigned_tasks = assigned_tasks
        self.tasks_completed = []
        self.resource = resource
        self.lambda_tasks = lambda_tasks
        self.lambda_dwell_time = lambda_dwell_time      #do sth with this
        self.generator = generator
        self.inROI = True   #is vehicle in the intersection/ region of interest

        print(f'Vehicle {self.id} spawned in environment at time {self.environment.now}')

        #start lifecycle of every task that is created before simulation
        for task in assigned_tasks:     
            self.environment.total_task_count += 1              
            self.environment.process(task.process(self.environment, resource))

        #start task generator
        self.environment.process(self.generateTasks())

    def scheduleDeparture(self):
        yield self.environment.timeout(np.random.poisson(self.lambda_dwell_time))
        self.inROI = False

    def generateTasks(self):

        self.environment.process(self.scheduleDeparture())

        while self.inROI:

            priority = random.randint(0, 5)
            duration = random.randint(1, 5)
            yield self.environment.timeout(np.random.poisson(self.lambda_tasks)) 
            
            #add task to vehicle
            new_task = Task(self.environment.total_task_count, priority, duration, self)
            self.assigned_tasks.append(new_task)
            self.environment.total_task_count += 1

            #activate tasks lifecycle 
            self.environment.process(new_task.process(self.environment, self.resource))

            print(f'Vehicle {self.id} generated a new Task at {self.environment.now} - new Task id:{new_task.id}, priority: {new_task.priority} and duration: {new_task.duration}')

        self.generator.vehicles_present.remove(self)
        self.generator.vehicles_departed.append(self)
        print(f"vehicle {self.id} left the intersection and stopped generating tasks at {self.environment.now}")





In [65]:
class VehicleGenerator:

    def __init__(self, lambda_vehicles, lambda_tasks, lambda_dwell_time, environment, resource):

        self.lambda_vehicles = lambda_vehicles
        self.lambda_tasks = lambda_tasks
        self.lambda_dwell_time = lambda_dwell_time
        self.environment = environment
        self.resource = resource

        self.vehicles = []
        self.vehicles_present = []
        self.vehicles_departed = []
        self.vehicle_count = 0
        self.total_tasks = self.countTasks()
        #self.task_count # make task count function

        environment.process(self.generateVehicles())

    def generateVehicles(self):

        while True:
             
             yield self.environment.timeout(np.random.poisson(self.lambda_vehicles))

             new_vehicle = Vehicle(self.vehicle_count, self.environment,[], self.resource, self.lambda_tasks, self.lambda_dwell_time, self)
             self.vehicle_count +=1 
             self.vehicles.append(new_vehicle)
             self.vehicles_present.append(new_vehicle)


    def countTasks(self):

        total_tasks = 0

        for vehicle in self.vehicles:
            total_tasks += len(vehicle.assigned_tasks)
            
        return total_tasks


In [66]:
intersection = simpy.Environment()
intersection.plan = []
intersection.total_task_count = 0
intersection.total_tasks_completed = 0

#different resources to try out for different scheduling types 
cpuEDF = simpy.PreemptiveResource(intersection, capacity = 1)
cpuFifo = simpy.Resource(intersection)

poissonVehicles = 5  # time interval per vehicle
poissonTasks = 2.5   # time interval per task

poisson_dwell_time  = 30

generator = VehicleGenerator(poissonVehicles, poissonTasks, poisson_dwell_time, intersection, cpuEDF)     #switch out cpu resource for different scheduling results.

intersection.run(until = 60)

Vehicle 0 spawned in environment at time 2
Vehicle 1 spawned in environment at time 6
Vehicle 0 generated a new Task at 7 - new Task id:0, priority: 5 and duration: 1
Task 0 came into existence at 7
Task 0 requesting at 7 with priority 5
Task 0 got resource at 7
Vehicle 2 spawned in environment at time 8
Vehicle 0 generated a new Task at 8 - new Task id:1, priority: 1 and duration: 2
Task 1 came into existence at 8
Task 1 requesting at 8 with priority 1
Task 0 got preempted by <Process(process) object at 0x1180f6590> at 8 after 1 time entities
Vehicle 0 generated a new Task at 8 - new Task id:2, priority: 5 and duration: 5
Task 2 came into existence at 8
Task 2 requesting at 8 with priority 5
Task 1 got resource at 8
Vehicle 1 generated a new Task at 9 - new Task id:3, priority: 0 and duration: 3
Task 3 came into existence at 9
Task 3 requesting at 9 with priority 0
Task 1 got preempted by <Process(process) object at 0x1180f7940> at 9 after 1 time entities
Task 1 requesting at 9 with p

In [67]:
print(f"Total task count: {intersection.total_task_count}")
print(f"Number of tasks completed: {intersection.total_tasks_completed}")
print(f"Number of vehicles generated: {len(generator.vehicles)}")
print(f"Number of vehicles in intersection: {len(generator.vehicles_present)}")
print(f"Number of departed vehicles: {len(generator.vehicles_departed)}")

for v in generator.vehicles:
    print(f"Vehicle {v.id} had {len(v.assigned_tasks)} tasks assigned and {len(v.tasks_completed)} tasks completed ")

print("Task scheduling plan:")

Total task count: 134
Number of tasks completed: 17
Number of vehicles generated: 14
Number of vehicles in intersection: 6
Number of departed vehicles: 8
Vehicle 0 had 11 tasks assigned and 2 tasks completed 
Vehicle 1 had 9 tasks assigned and 2 tasks completed 
Vehicle 2 had 12 tasks assigned and 3 tasks completed 
Vehicle 3 had 15 tasks assigned and 2 tasks completed 
Vehicle 4 had 21 tasks assigned and 2 tasks completed 
Vehicle 5 had 8 tasks assigned and 1 tasks completed 
Vehicle 6 had 14 tasks assigned and 1 tasks completed 
Vehicle 7 had 11 tasks assigned and 2 tasks completed 
Vehicle 8 had 11 tasks assigned and 1 tasks completed 
Vehicle 9 had 7 tasks assigned and 0 tasks completed 
Vehicle 10 had 6 tasks assigned and 1 tasks completed 
Vehicle 11 had 5 tasks assigned and 0 tasks completed 
Vehicle 12 had 4 tasks assigned and 0 tasks completed 
Vehicle 13 had 0 tasks assigned and 0 tasks completed 
Task scheduling plan:
